# Laboratorio 04 | The Games of Nodes Part II

Agregaciones en Neo4j (Cypher) usando Count, Collect y Size

## Configuración del entorno

In [3]:
from neo4j import GraphDatabase

uri = "<URI>"         
username = "<USERNAME>"                      
password = "<PASSWORD>"

driver = GraphDatabase.driver(uri, auth=(username, password))

def close_connection():
    driver.close()

def verify_connection():
    try:
        with driver.session() as session:
            result = session.run("RETURN 1 AS value")
            value = result.single()["value"]
            print("Connection successful. Result:", value)
    except Exception as e:
        print("Connection failed:", e)

### Verificar la conexión

In [4]:
verify_connection()
close_connection()

Connection successful. Result: 1
